In [8]:
from Graphs import MyGraph
import unittest

## Genome Assembly

### Conceptual Overview

**Purpose:**  
Genome assembly aims to reconstruct the original DNA sequence from a collection of overlapping fragments (k-mers), as produced by modern sequencing technologies.

**Main Strategies:**

1. **De Bruijn Graphs:**

   - Each k-mer is represented as a directed edge.
   - The nodes of the graph are all possible (k-1)-length prefixes and suffixes of the k-mers.
   - Assembly involves finding an **Eulerian path**—a path that traverses every edge exactly once.
   - The original sequence is reconstructed by following this path and concatenating the appropriate characters from each node.

2. **Overlap Graphs:**

   - Each k-mer is represented as a node.
   - An edge is drawn from node A to node B if the suffix of A (length k-1) matches the prefix of B.
   - The goal is to find a **Hamiltonian path**—a path that visits every node exactly once.
   - The sequence is assembled by joining the fragments along this path, using the overlaps to avoid redundancy.

**Comparison:**

- De Bruijn graphs are highly efficient for large datasets and handle repetitive regions well, making them the backbone of most modern assemblers.
- Overlap graphs are conceptually straightforward but computationally intensive for large numbers of fragments, as finding Hamiltonian paths is a hard problem.

---

### Implementation Details

**De Bruijn Graph Assembly:**

- For each k-mer, identify its prefix (first k-1 bases) and suffix (last k-1 bases).
- Add a directed edge from the prefix to the suffix.
- The resulting graph should be nearly balanced: one node with one extra outgoing edge (start) and one with one extra incoming edge (end).
- Temporarily add an edge to close the cycle, then use Hierholzer’s algorithm to find an Eulerian cycle.
- Remove the temporary edge to obtain the actual Eulerian path.
- Reconstruct the sequence by concatenating the first node and the last character of each subsequent node in the path.

**Overlap Graph Assembly:**

- Treat each fragment as a unique node (even if sequences repeat, assign unique identifiers).
- For every pair of fragments, create an edge if the suffix of one matches the prefix of another.
- Use backtracking to search for a Hamiltonian path—a path that visits each node exactly once.
- The final sequence is built by taking the full first fragment and, for each subsequent fragment in the path, appending only the last character.

**Practical Considerations:**

- De Bruijn graph methods are preferred for real-world genome assembly due to their scalability and robustness to errors and repeats.
- Overlap graphs are useful for illustrating the assembly concept but are less practical for large-scale datasets due to computational complexity.

In [9]:
def get_prefix(seq):
    """Return the prefix (all but last character) of a sequence."""
    return seq[:-1]

def get_suffix(seq):
    """Return the suffix (all but first character) of a sequence."""
    return seq[1:]

class DeBruijnGraph(MyGraph):
    """
    De Bruijn graph for genome assembly from k-mers.
    Nodes: (k-1)-mers; Edges: k-mers as transitions.
    """

    def __init__(self, kmers):
        super().__init__()
        for kmer in kmers:
            self.add_edge(get_prefix(kmer), get_suffix(kmer), 1)  # Peso 1

    def find_eulerian_path(self):
        """Find an Eulerian path if the graph is nearly balanced."""
        start, end = self._find_path_ends()
        if not start or not end:
            return None

        self.add_edge(end, start, 1)  # Peso 1 (temporário)

        # Use stack-based Hierholzer's algorithm
        path, stack = [], [start]
        local_edges = {u: list(vs) for u, vs in self.graph.items()}
        while stack:
            u = stack[-1]
            if local_edges.get(u):
                stack.append(local_edges[u].pop()[0])  # Só o destino
            else:
                path.append(stack.pop())
        path.reverse()

        # Remove the temporary edge from the path
        for i in range(len(path) - 1):
            if path[i] == end and path[i + 1] == start:
                return path[i + 1:] + path[1:i + 1]
        return None

    def _find_path_ends(self):
        """Return (start, end) nodes for Eulerian path if nearly balanced."""
        start = end = None
        for node in self.graph:
            indeg = self.in_degree(node)
            outdeg = self.out_degree(node)
            if outdeg - indeg == 1:
                if start is None:
                    start = node
                else:
                    return None, None
            elif indeg - outdeg == 1:
                if end is None:
                    end = node
                else:
                    return None, None
            elif indeg != outdeg:
                return None, None
        return start, end

    def assemble_sequence(self, path):
        """Reconstruct sequence from Eulerian path."""
        if not path:
            return None
        return path[0] + ''.join(node[-1] for node in path[1:])

class OverlapGraph(MyGraph):
    """
    Overlap graph for genome assembly from k-mers.
    Nodes: uniquely labeled k-mers; Edges: overlap of k-1 between suffix and prefix.
    """

    def __init__(self, kmers):
        super().__init__()
        self.labeled = [f"{kmer}#{i}" for i, kmer in enumerate(kmers)]
        for label in self.labeled:
            self.add_vertex(label)
        for src in self.labeled:
            src_seq = src.split('#')[0]
            for dst in self.labeled:
                if src != dst and get_suffix(src_seq) == get_prefix(dst.split('#')[0]):
                    self.add_edge(src, dst, 1)  # Peso 1

    def find_hamiltonian_path(self):
        """Find a Hamiltonian path using DFS with pruning."""
        def dfs(path, visited):
            if len(path) == len(self.graph):
                return path
            for neighbor, _ in self.graph[path[-1]]:
                if neighbor not in visited:
                    res = dfs(path + [neighbor], visited | {neighbor})
                    if res:
                        return res
            return None

        for start in self.graph:
            res = dfs([start], {start})
            if res:
                return res
        return None

    def _extract_seq(self, label):
        return label.split('#')[0]

    def assemble_sequence(self, path):
        """Reconstruct sequence from Hamiltonian path."""
        if not path:
            return None
        return self._extract_seq(path[0]) + ''.join(self._extract_seq(n)[-1] for n in path[1:])

In [10]:
def generate_kmers(seq, k):
    """Generate all k-mers from a sequence (no sorting, preserves order)."""
    return [seq[i:i+k] for i in range(len(seq) - k + 1)]

def genome_assembly_all(seq, k):
    print("Original sequence:", seq)
    kmers = generate_kmers(seq, k)
    print("k-mers:", kmers)

    # De Bruijn Graph Assembly
    print("\n--- De Bruijn Graph Assembly ---")
    dbg = DeBruijnGraph(kmers)  # Usa MyGraph como base
    dbg.print_graph()
    path = dbg.find_eulerian_path()
    if path:
        print("Eulerian path found:", path)
        print("Assembled sequence:", dbg.assemble_sequence(path))
    else:
        print("Eulerian path not found.")

    # Overlap Graph Assembly
    print("\n--- Overlap Graph Assembly ---")
    og = OverlapGraph(kmers)  # Usa MyGraph como base
    og.print_graph()
    path = og.find_hamiltonian_path()
    if path:
        print("Hamiltonian path found:", path)
        print("Assembled sequence:", og.assemble_sequence(path))
    else:
        print("Hamiltonian path not found.")

# Exemplo de utilização:
genome_assembly_all('GATTACAGATTACAGGATCAGATTACA', 4)

Original sequence: GATTACAGATTACAGGATCAGATTACA
k-mers: ['GATT', 'ATTA', 'TTAC', 'TACA', 'ACAG', 'CAGA', 'AGAT', 'GATT', 'ATTA', 'TTAC', 'TACA', 'ACAG', 'CAGG', 'AGGA', 'GGAT', 'GATC', 'ATCA', 'TCAG', 'CAGA', 'AGAT', 'GATT', 'ATTA', 'TTAC', 'TACA']

--- De Bruijn Graph Assembly ---
AGT#0 -> []
GTT#1 -> []
TTC#2 -> []
ATTA#0 -> []
TTAC#1 -> []
TACC#2 -> []
ACCG#3 -> []
AAA#0 -> []
CCC#1 -> []
GGG#2 -> []
GAT -> [('ATT', 1), ('ATT', 1), ('ATC', 1), ('ATT', 1)]
ATT -> [('TTA', 1), ('TTA', 1), ('TTA', 1)]
TTA -> [('TAC', 1), ('TAC', 1), ('TAC', 1)]
TAC -> [('ACA', 1), ('ACA', 1), ('ACA', 1)]
ACA -> [('CAG', 1), ('CAG', 1)]
CAG -> [('AGA', 1), ('AGG', 1), ('AGA', 1)]
AGA -> [('GAT', 1), ('GAT', 1)]
AGG -> [('GGA', 1)]
GGA -> [('GAT', 1)]
ATC -> [('TCA', 1)]
TCA -> [('CAG', 1)]
Eulerian path found: ['GAT', 'ATC', 'TCA', 'CAG', 'AGA', 'GAT', 'ATT', 'TTA', 'TAC', 'ACA', 'CAG', 'AGG', 'GGA', 'GAT', 'ATT', 'TTA', 'TAC', 'ACA', 'CAG', 'AGA', 'GAT', 'ATT', 'TTA', 'TAC', 'ACA']
Assembled sequence: G

In [11]:
class TestGenomeAssembly(unittest.TestCase):
    # --- De Bruijn Graph Tests ---

    def test_generate_kmers_basic(self):
        self.assertEqual(
            generate_kmers("ATGC", 3),
            ["ATG", "TGC"]
        )

    def test_generate_kmers_with_repeats(self):
        self.assertEqual(
            generate_kmers("ATGATG", 3),
            ["ATG", "TGA", "GAT", "ATG"]
        )

    def test_debruijn_graph_nodes_and_edges(self):
        kmers = ["GAT", "ATT", "TTA", "TAG"]
        dbg = DeBruijnGraph(kmers)
        expected_nodes = {"GA", "AT", "TT", "TA"}
        self.assertTrue(expected_nodes.issubset(set(dbg.graph.keys())))
        self.assertIn(("AT", 1), dbg.graph["GA"])
        self.assertIn(("TA", 1), dbg.graph["TT"])

    def test_debruijn_eulerian_path_and_assembly(self):
        kmers = ["CTTA", "ACCA", "TACC", "GGCT", "GCTT", "TTAC"]
        dbg = DeBruijnGraph(kmers)
        path = dbg.find_eulerian_path()
        if path is not None:
            seq = dbg.assemble_sequence(path)
            for kmer in kmers:
                self.assertIn(kmer, seq)
        else:
            self.assertIsNone(path)

    def test_debruijn_no_eulerian_path(self):
        kmers = ["AAA", "CCC", "GGG"]
        dbg = DeBruijnGraph(kmers)
        path = dbg.find_eulerian_path()
        self.assertIsNone(path)

    def test_debruijn_assemble_sequence_none(self):
        dbg = DeBruijnGraph([])
        self.assertIsNone(dbg.assemble_sequence(None))

    # --- Overlap Graph Tests ---

    def test_overlap_graph_hamiltonian_and_assembly(self):
        frags = ["ATTA", "TTAC", "TACC", "ACCG"]
        og = OverlapGraph(frags)
        path = og.find_hamiltonian_path()
        if path is not None:
            seq = og.assemble_sequence(path)
            for frag in frags:
                self.assertIn(frag, seq)
        else:
            self.assertIsNone(path)

    def test_overlap_graph_no_hamiltonian(self):
        frags = ["AAA", "CCC", "GGG"]
        og = OverlapGraph(frags)
        self.assertIsNone(og.find_hamiltonian_path())

    def test_overlap_assemble_sequence_none(self):
        og = OverlapGraph([])
        self.assertIsNone(og.assemble_sequence(None))

In [12]:
run_tests()

test_debruijn_assemble_sequence_none (__main__.TestGenomeAssembly.test_debruijn_assemble_sequence_none) ... ok
test_debruijn_eulerian_path_and_assembly (__main__.TestGenomeAssembly.test_debruijn_eulerian_path_and_assembly) ... ok
test_debruijn_graph_nodes_and_edges (__main__.TestGenomeAssembly.test_debruijn_graph_nodes_and_edges) ... ok
test_debruijn_no_eulerian_path (__main__.TestGenomeAssembly.test_debruijn_no_eulerian_path) ... ok
test_generate_kmers_basic (__main__.TestGenomeAssembly.test_generate_kmers_basic) ... ok
test_generate_kmers_with_repeats (__main__.TestGenomeAssembly.test_generate_kmers_with_repeats) ... ok
test_overlap_assemble_sequence_none (__main__.TestGenomeAssembly.test_overlap_assemble_sequence_none) ... ok
test_overlap_graph_hamiltonian_and_assembly (__main__.TestGenomeAssembly.test_overlap_graph_hamiltonian_and_assembly) ... ok
test_overlap_graph_no_hamiltonian (__main__.TestGenomeAssembly.test_overlap_graph_no_hamiltonian) ... ok

------------------------------